# Введение в искусственные нейронные сети
# Урок 5. Рекуррентные нейронные сети

## Практическое задание

<ol>
    <li>Попробуйте изменить параметры нейронной сети работающей с датасетом imdb так, чтобы улучшить ее точность. Приложите анализ.</li>
    <li>Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.</li>
    <li>* Попробуйте на numpy реализовать нейронную сеть архитектуры LSTM</li>
    <li>* Предложите свои варианты решения проблемы исчезающего градиента в RNN</li>
</ol>

### Нейронная сеть работающая с датасетом imdb

In [95]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, LSTM, SimpleRNN
from keras.datasets import imdb

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
tf.random.set_seed(48)

max_features = 5000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 200
batch_size = 50 # увеличьте значение для ускорения обучения

print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

print('Pad последовательности (примеров в x единицу времени)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0, recurrent_dropout=0))
model.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=2, # увеличьте при необходимости
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)


Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности
Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 200)
x_test shape: (25000, 200)
Построение модели...
Процесс обучения...
Epoch 1/2
500/500 [==============================] - 9s 17ms/step - loss: 0.4136 - accuracy: 0.8120 - val_loss: 0.3403 - val_accuracy: 0.8538
Epoch 2/2
500/500 [==============================] - 2s 5ms/step - loss: 0.3037 - accuracy: 0.8740
Результат при тестировании: 0.3037417531013489
Тестовая точность: 0.8740000128746033


### Нейронная сеть генерирующая текст

In [1]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential

In [2]:
# построчное чтение из примера с текстом 
#with open("alice_in_wonderland.txt", 'rb') as _in:
with open("SSSR_anekdot.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("utf-8", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)

In [3]:
# Удалим неинформативные строки
text = text.replace(" xxx", "") 
text = text.replace(" ххх", "") 
text = text.replace("  хх", "") 
text = text.replace("   х", "") 
text = text.replace(" XXX", "") 
text = text.replace("  - ", "") 
text = text.replace("  – ", "") 
text[:1000]

'xxx Партия-бублик, народу-дырка от бублика! Это и есть советская республика! -\xa0 Что такое РСФСР? -\xa0 Редкий случай феноменального сумасшествия России. –\xa0Скажите, вы коммунист? –\xa0Нет, я сочувствующий. Но помочь ничем не могу! Муж возвращается с работы: –\xa0Маша, я в партию вступил! Жена ворчит: –\xa0Вечно ты во что-нибудь вступаешь!.. -Что такое РКП/б/? -\xa0 Россия кончит погромом. –АВКП/б/? -\xa0 Все кончится погромом. -\xa0 Ну, а "б" в скобках? -\xa0 Большим погромом! Крестьянин приехал в город, заходит в магазин и спрашивает: –\xa0Нет ли у вас вожжей ? Продавец, не расслышав, показывает тому портреты вождей. Крестьянин замахал руками: –\xa0Да нет, мне нужны настоящие, крепкие! хх Коммунизм – это советская власть плюс электрификация. Отсюда следует: советская власть – это коммунизм минус электрификация. Или : электрификация – это коммунизм минус советская власть. Из репродуктора доносится: " Великая Октябрьская революция навеки освободила народ от цепей капитализма". Ста

In [30]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
tf.random.set_seed(48)

chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапараметров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 1128, 512 #128, 128
NUM_ITERATIONS = 3 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 10 #1
NUM_PREDS_PER_EPOCH = 200 #100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. 
За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в 
вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию 
ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=True,
        input_shape=(SEQLEN, nb_chars),
        #dropout=0.2, 
        #unroll=True
    )
)
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=True,
        input_shape=(SEQLEN, nb_chars),
        #dropout=0.2, 
        #unroll=True
    )
)
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        #dropout=0.2, 
        #unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam") #optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
Epoch 1/10
227/227 [==============================] - 8s 33ms/step - loss: 2.8342
Epoch 2/10
227/227 [==============================] - 8s 34ms/step - loss: 2.2314
Epoch 3/10
227/227 [==============================] - 8s 36ms/step - loss: 1.9257
Epoch 4/10
227/227 [==============================] - 8s 34ms/step - loss: 1.7113
Epoch 5/10
227/227 [==============================] - 7s 33ms/step - loss: 1.5559
Epoch 6/10
227/227 [==============================] - 8s 34ms/step - loss: 1.4253
Epoch 7/10
227/227 [==============================] - 8s 34ms/step - loss: 1.3046
Epoch 8/10
227/227 [==============================] - 8s 34ms/step - loss: 1.1825
Epoch 9/10
227/227 [==============================] - 7s 33ms/step - loss: 1.0604
Epoch 10/10
227/227 [==============================] - 8s 35ms/step - loss: 0.9392
Генерация из посева: звыходного
звыходного разных сломах из СССР. -  А что это вы можете советские вожди, который бог после прихода в своих коммунизме разворуют в по

### Выводы

1. В отличие от предыдущего ДЗ увеличение количества эпох в данном примере с датасетом imdb не привело к положительному результату. Напротив, даже при небольшом увеличении эпох (до 2-3) возникает переобучение модели. 
Изменение оптимизатора, простое увеличение числа нейронов в слое и добавление новых слоёв LSTM, так же не привело 
к значимому эффекту. Попытка бороться с переобучением с помощью увеличения dropout и recurrent_dropout так же окончилась неудачно. Зато обнуление параметров dropout и recurrent_dropout слегка увеличило точность.
Так же помогло увеличить точность изменение параметров обучаемого текста. В частности: увеличение длины текста 
maxlen с 80 до 200 слов и уменьшение max_features с 20000 до 5000. Плюс слегка увеличил число эпох с 1 до 2. 
В результате точность увеличилась с до 0.8342 до 0.8740.
<br>
2. Для генерации текста использовал собрание анекдотов СССР. Для того чтобы анектод получался более законченный увеличил длину генерируемого текста до 200 символов. Так же пришлось сделать предобработку текста, удалив неинформативные часто повторяющиеся символы, т.к. модель очень сильно за них "цеплялась". Сначала текст получался состоящим из часто повторяющихся слов. Увеличил число итераций до 3, число эпох до 10, число слоёв GRU до 3-х и число нейронов в слоях до 512 - это значительно улучшило текст, текст получился более-менее осмысленный. Так же для увеличения скорости обучения увеличил BATCH_SIZE до 1128. Оценивать задание с генерацией текста оказалось сложнее, в том смысле, что несмотря на минимальную ошибку (loss), понятность генерируемого текста всё равно приходится оценивать визуально.<br>
Самый лучший сгенерированный текст:<br>
такая маленькая юбка. –А миникомпьютер? -  Это когда все спят! – В это время празднеств по поводу трехсотлетия воссоединения Украины с Россией , не успел надеть штаны: так и пошел в одном исподтерке отделение 